In [1]:
# An implementaion of how to display all of the exchanges and market data sources that 
# are available on the CCXT trading library.

import ccxt
print(ccxt.exchanges)

['_1broker', '_1btcxe', 'acx', 'allcoin', 'anxpro', 'anybits', 'bibox', 'binance', 'bit2c', 'bitbank', 'bitbay', 'bitfinex', 'bitfinex2', 'bitflyer', 'bithumb', 'bitkk', 'bitlish', 'bitmarket', 'bitmex', 'bitsane', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitz', 'bl3p', 'bleutrade', 'braziliex', 'btcbox', 'btcchina', 'btcexchange', 'btcmarkets', 'btctradeim', 'btctradeua', 'btcturk', 'btcx', 'bxinth', 'ccex', 'cex', 'chbtc', 'chilebit', 'cobinhood', 'coinbase', 'coinbasepro', 'coincheck', 'coinegg', 'coinex', 'coinexchange', 'coinfalcon', 'coinfloor', 'coingi', 'coinmarketcap', 'coinmate', 'coinnest', 'coinone', 'coinsecure', 'coinspot', 'cointiger', 'coolcoin', 'crypton', 'cryptopia', 'deribit', 'dsx', 'ethfinex', 'exmo', 'exx', 'fcoin', 'flowbtc', 'foxbit', 'fybse', 'fybsg', 'gatecoin', 'gateio', 'gdax', 'gemini', 'getbtc', 'hadax', 'hitbtc', 'hitbtc2', 'huobi', 'huobicny', 'huobipro', 'ice3x', 'independentreserve', 'indodax', 'itbit', 'jubi', 'kraken', 'kucoin', 'kuna', 'lakebt

In [43]:
# An example of how to instantiate and load an exchange to see all of the tickers available for 
# trade on the exchange. 

import ccxt

exchange = ccxt.gemini()
markets = exchange.load_markets() 
symbols = exchange.symbols


print(exchange.id, symbols)

gemini ['BTC/USD', 'ETH/BTC', 'ETH/USD', 'ZEC/BTC', 'ZEC/ETH', 'ZEC/USD']


In [46]:
# implementation of bid, ask and spread data based on exchanges generated from 
# the ccxt trading library system; specifaclly 'Gemini Exchange'

import ccxt
import time

exchange = ccxt.gemini()
exchange.load_markets()
print('Rate Limit for Gemini Exchange: ', exchange.rateLimit)
delay_seconds = exchange.rateLimit/ 1000
symbols = dict()


for symbol in exchange.markets: 

    start_time = time.clock() 
    orderbook = exchange.fetch_order_book(symbol)
    bid = orderbook['bids'][0][0] if len(orderbook['bids']) > 0 else None
    ask = orderbook['asks'][0][0] if len(orderbook['asks']) > 0 else None
    spread = (ask - bid) if (bid and ask) else None
    market_price = {
        'bid' : bid, 
        'ask' : ask, 
        'spread' : spread
    }
    
    symbols[symbol] = market_price
    print(exchange.id, symbol, 'market price:\n', market_price)
    time.sleep(delay_seconds - (time.clock() - start_time))

Rate Limit for Gemini Exchange:  1500
gemini BTC/USD market price:
 {'bid': 6594.61, 'ask': 6598.83, 'spread': 4.220000000000255}
gemini ETH/BTC market price:
 {'bid': 0.0712, 'ask': 0.07139, 'spread': 0.00018999999999999573}
gemini ETH/USD market price:
 {'bid': 469.38, 'ask': 470.2, 'spread': 0.8199999999999932}
gemini ZEC/USD market price:
 {'bid': 174.08, 'ask': 175.0, 'spread': 0.9199999999999875}
gemini ZEC/BTC market price:
 {'bid': 0.02642, 'ask': 0.02651, 'spread': 8.99999999999998e-05}
gemini ZEC/ETH market price:
 {'bid': 0.3637, 'ask': 0.3785, 'spread': 0.01479999999999998}


## CrypotExchange Data Downloader 

The following code was used to parse a specific exchange and download the data necessary for the analysis that takes place in the suceeding cell. Further specifics on the code snippet can be explored from the housing [github repository](https://github.com/Jzbonner/Python-for-Data-Science) 

```python 
import ccxt 
from datetime import datetime, timedelta, timezone 
import math
import argparse 
import pandas as pd 

def parse_args(): 
    parser = argparse.ArgumentParser(description = 'CCXT Market Downloader')

    parser.add_argument('-s','--symbol',
                        type=str,
                        required=True,
                        help='The Symbol of the Instrument/Currency Pair To Download')

    parser.add_argument('-e','--exchange',
                        type=str,
                        required=True,
                        help='The exchange to download from')
 
    parser.add_argument('-t','--timeframe',
                        type=str,
                        default='1d',
                        choices=['1m', '5m','15m', '30m','1h', '2h', '3h', '4h', '6h', '12h', '1d', '1M', '1y'],
                        help='The timeframe to download')
 
 
    parser.add_argument('--debug',
                            action ='store_true',
                            help=('Print Sizer Debugs'))
 
    return parser.parse_args()

args = parse_args() 

try: 
    exchange = getattr (ccxt, args.exchange) ()
except AttributeError:
    print('-'*36,' ERROR ','-'*35)
    print('Exchange "{}" not found. Please check the exchange is supported.'.format(args.exchange))
    print('-'*80)
    quit()

if exchange.has["fetchOHLCV"] == False:
    print('-'*36,' ERROR ','-'*35)
    print('{} does not support fetching OHLC data. Please use another exchange'.format(args.exchange))
    print('-'*80)
    quit()

if args.timeframe not in exchange.timeframes:
    print('-'*36,' ERROR ','-'*35)
    print('The requested timeframe ({}) is not available from {}\n'.format(args.timeframe,args.exchange))
    print('Available timeframes are:')
    for key in exchange.timeframes.keys():
        print('  - ' + key)
    print('-'*80)
    quit()

exchange.load_markets()
if args.symbol not in exchange.symbols:
    print('-'*36,' ERROR ','-'*35)
    print('The requested symbol ({}) is not available from {}\n'.format(args.symbol,args.exchange))
    print('Available symbols are:')
    for key in exchange.symbols:
        print('  - ' + key)
    print('-'*80)
    quit()

data = exchange.fetch_ohlcv(args.symbol, args.timeframe)
header = ['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.DataFrame(data, columns=header).set_index('TimeStamp')

symbol_out = args.symbol.replace("/","")
filename = '{}-{}-{}.csv'.format(args.exchange, symbol_out,args.timeframe)
df.to_csv(filename)
```

In [63]:
# Implementation of Bokeh and Pandas to display NEO/ETH order book OHLCV trends from September 2017 to July 2018. 
# CSV data was taken from the Binance Exchange API and is logged on a 1-day time frame. 

%matplotlib inline

from datetime import datetime 
from bokeh.plotting import figure, output_notebook, show
output_notebook()

import matplotlib
import matplotlib.pyplot as plt
import ccxt 
import numpy as np
import pandas as pd 


data = pd.read_csv("PythonScripts/binance-NEOETH-1d.csv")
data['Date'] = pd.to_datetime(data['TimeStamp'], unit='ms')

j = figure(
        plot_width=800, 
        plot_height=250, 
        title='Visual Representation for NEO/ETH Close Values on Binance', 
        x_axis_label='Date',
        y_axis_label='Close for NEO/ETH'
    )

j.line(data['Date'], data['Close'], color='navy', legend="NEO/ETH", line_width=2)

show(j)
data.head(n=10)


Loading BokehJS ...

TimeStamp      Open      High       Low     Close    Volume       Date
0  1506556800000  0.097000  0.110000  0.055000  0.109997   3863.12 2017-09-28
1  1506643200000  0.106969  0.106969  0.092000  0.103726   5850.06 2017-09-29
2  1506729600000  0.102201  0.120001  0.096200  0.112870  10078.76 2017-09-30
3  1506816000000  0.112868  0.119875  0.096200  0.119869   5917.43 2017-10-01
4  1506902400000  0.117001  0.129999  0.115103  0.123963  12633.78 2017-10-02
5  1506988800000  0.122100  0.129890  0.109657  0.114926   7872.11 2017-10-03
6  1507075200000  0.114926  0.120890  0.101050  0.109599  12567.92 2017-10-04
7  1507161600000  0.109589  0.119899  0.100090  0.109001  12798.58 2017-10-05
8  1507248000000  0.113599  0.116200  0.108005  0.108500  15187.89 2017-10-06
9  1507334400000  0.108031  0.119000  0.106302  0.116000  19731.76 2017-10-07

In [86]:
# Loads 6 US servicing cryptocurrency exchanges and their respective tickers

import ccxt

crypto_exchanges = (
    ccxt.bittrex(),
    ccxt.binance(), 
    ccxt.gemini(),
    ccxt.coingi(), 
    ccxt.gdax(), 
    ccxt.okcoinusd() 
)

for exchange in crypto_exchanges: 
    markets = exchange.load_markets() 
    symbols = exchange.symbols
    print(exchange.id, symbols)


bittrex ['2GIVE/BTC', 'ABY/BTC', 'ADA/BTC', 'ADA/ETH', 'ADA/USDT', 'ADT/BTC', 'ADT/ETH', 'ADX/BTC', 'ADX/ETH', 'AEON/BTC', 'AID/BTC', 'AID/ETH', 'AMP/BTC', 'ANT/BTC', 'ANT/ETH', 'ARDR/BTC', 'ARK/BTC', 'AUR/BTC', 'BAT/BTC', 'BAT/ETH', 'BAY/BTC', 'BCH/BTC', 'BCH/ETH', 'BCH/USDT', 'BCPT/BTC', 'BCPT/ETH', 'BCY/BTC', 'BITB/BTC', 'BKX/BTC', 'BLK/BTC', 'BLOCK/BTC', 'BLT/BTC', 'BLT/ETH', 'BNT/BTC', 'BNT/ETH', 'BRK/BTC', 'BRX/BTC', 'BSD/BTC', 'BTC/USD', 'BTC/USDT', 'BTG/BTC', 'BTG/ETH', 'BTG/USDT', 'BURST/BTC', 'BYC/BTC', 'CANN/BTC', 'CBC/BTC', 'CFI/BTC', 'CFI/ETH', 'CLAM/BTC', 'CLOAK/BTC', 'CMCT/BTC', 'COVAL/BTC', 'CRB/BTC', 'CRW/BTC', 'CURE/BTC', 'CVC/BTC', 'CVC/ETH', 'DASH/BTC', 'DASH/ETH', 'DASH/USDT', 'DCR/BTC', 'DCR/USDT', 'DCT/BTC', 'DGB/BTC', 'DGB/ETH', 'DMD/BTC', 'DMT/BTC', 'DMT/ETH', 'DNT/BTC', 'DNT/ETH', 'DOGE/BTC', 'DOPE/BTC', 'DTB/BTC', 'DYN/BTC', 'EBST/BTC', 'EDG/BTC', 'EFL/BTC', 'EGC/BTC', 'EMC/BTC', 'EMC2/BTC', 'ENG/BTC', 'ENG/ETH', 'ENRG/BTC', 'ERC/BTC', 'ETC/BTC', 'ETC/ETH', '

In [ ]:
%matplotlib inline

from datetime import datetime 
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

import matplotlib
import matplotlib.pyplot as plt
import ccxt 
import numpy as np
import pandas as pd 

output_notebook() 